In [2]:
import pandas as pd
import numpy as np 
import sklearn


### Reading and Preprocessing data

In [3]:
# Reading census dataset 
train_csv=pd.read_csv("census.csv")
test_csv=pd.read_csv("test_census.csv")

train_csv.drop(["race","sex","education_level"],axis=1)
test_csv.drop(["race","sex","education_level","Unnamed: 0"],axis=1)

,age,workclass,education-num,marital-status,occupation,relationship,capital-gain,capital-loss,hours-per-week,native-country
0,21.0,Private,6.0,Married-civ-spouse,Craft-repair,Husband,0.0,0.0,40.0,United-States
1,49.0,Private,13.0,Married-civ-spouse,Adm-clerical,Wife,0.0,0.0,40.0,United-States
2,44.0,Self-emp-not-inc,12.0,Married-civ-spouse,Other-service,Wife,0.0,0.0,99.0,United-States
3,34.0,Private,13.0,Married-civ-spouse,Sales,Husband,7298.0,0.0,46.0,United-States
4,24.0,Private,9.0,Married-civ-spouse,Machine-op-inspct,Husband,0.0,0.0,40.0,United-States
...,...,...,...,...,...,...,...,...,...,...
45217,17.0,Private,7.0,Never-married,Other-service,Own-child,0.0,0.0,15.0,United-States
45218,50.0,Federal-gov,10.0,Never-married,Adm-clerical,Not-in-family,0.0,0.0,40.0,United-States
45219,17.0,Self-emp-inc,8.0,Never-married,Adm-clerical,Own-child,0.0,0.0,16.0,United-States
45220,38.0,Private,13.0,Married-civ-spouse,Prof-specialty,Husband,7688.0,0.0,40.0,United-States


In [ ]:
def preprocess(df):
    column = ["workclass","education-num","marital-status","occupation","relationship","native-country"]
    for col in column:
        Unique_values = list(df[col].unique())
        for x in len(Unique_values):


In [4]:
train_csv['A'] = train_csv['A'].map().fillna(train_csv['A'])

KeyError: 'A'